# Explore the AUG workflow and single flux tube UQ data sets

- this uses 
  - AUG_gem0_inoutput.txt
  - AUG_gem0_flux_area.txt

## A fairly standard set of imports

In [1]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
pd.options.display.max_rows = 128
from utility import _ewma, plot_smoothed, plot_unsmoothed

## Read and examine the workflow data

In [2]:
# read the data from the AUG workflow runs
AUG_gem0_wf = pd.read_table('../../data/AUG_gem0_inoutput.txt', delimiter='  *', engine='python')
AUG_gem0_pb = pd.read_table('../../data/AUG_gem0_flux_area.txt', delimiter='  *', engine='python') 

In [3]:
AUG_gem0_wf.describe().T

,count,mean,std,min,25%,50%,75%,max
time,101.0,1.505000e+00,2.930017e-01,1.005000e+00,1.255000e+00,1.505000e+00,1.755000e+00,2.005000e+00
Te-ft1,101.0,1.665552e+03,3.056173e+01,1.658569e+03,1.660164e+03,1.660613e+03,1.660829e+03,1.931695e+03
Te-ft2,101.0,1.474804e+03,2.663225e+01,1.468629e+03,1.470071e+03,1.470517e+03,1.470768e+03,1.709869e+03
Te-ft3,101.0,1.279224e+03,2.223527e+01,1.273794e+03,1.275173e+03,1.275642e+03,1.275943e+03,1.476204e+03
Te-ft4,101.0,1.081034e+03,1.697367e+01,1.076142e+03,1.077717e+03,1.078237e+03,1.078643e+03,1.226586e+03
Te-ft5,101.0,8.806221e+02,1.078277e+01,8.762250e+02,8.781232e+02,8.787603e+02,8.792701e+02,9.555430e+02
Te-ft6,101.0,6.764764e+02,9.297405e+00,6.549011e+02,6.748643e+02,6.757030e+02,6.763901e+02,7.626297e+02
Te-ft7,101.0,4.649402e+02,1.866597e+01,3.199812e+02,4.646810e+02,4.656842e+02,4.664656e+02,5.679306e+02
Te-ft8,101.0,2.333695e+02,9.264984e+00,1.626966e+02,2.330005e+02,2.339914e+02,2.346138e+02,2.840931e+02
dTe-ft1,101.0,-1.286055e+03,2.750300e+01,-1.489578e+03,-1.281751e+03,-1.281627e+03,-1.281420e+03,-1.269965e+03


In [4]:
AUG_gem0_pb.describe().T

,count,mean,std,min,25%,50%,75%,max
time,101.0,1.505000e+00,0.293002,1.005000e+00,1.255000e+00,1.505000e+00,1.755000e+00,2.005000e+00
Te-source-int-ft1,101.0,1.295628e+05,505.159089,1.285231e+05,1.295396e+05,1.295476e+05,1.295560e+05,1.343843e+05
Te-source-int-ft2,101.0,5.538644e+05,1916.238001,5.496102e+05,5.537879e+05,5.538239e+05,5.538613e+05,5.717929e+05
Te-source-int-ft3,101.0,1.016853e+06,3213.172472,1.005760e+06,1.016749e+06,1.016818e+06,1.016890e+06,1.045593e+06
Te-source-int-ft4,101.0,1.443716e+06,4760.076188,1.422219e+06,1.443577e+06,1.443684e+06,1.443788e+06,1.484127e+06
Te-source-int-ft5,101.0,1.807407e+06,7492.521090,1.776199e+06,1.807143e+06,1.807284e+06,1.807426e+06,1.873140e+06
Te-source-int-ft6,101.0,2.104540e+06,14054.825730,2.065229e+06,2.103986e+06,2.104163e+06,2.104352e+06,2.236045e+06
Te-source-int-ft7,101.0,2.347919e+06,39898.309252,2.086688e+06,2.347735e+06,2.347984e+06,2.348277e+06,2.639982e+06
Te-source-int-ft8,101.0,2.588873e+06,104917.550786,1.737899e+06,2.591367e+06,2.591898e+06,2.592655e+06,3.153085e+06
Ti-source-int-ft1,101.0,1.293146e+05,1756.886228,1.289811e+05,1.289894e+05,1.289982e+05,1.290068e+05,1.446005e+05


### Set up some column selections

In [5]:
active_t = []
active_t.append(['time', 'Te-ft1', 'dTe-ft1', 'Ti-ft1', 'dTi-ft1', 'flux-Te-ft1', 'flux-Ti-ft1'])
active_t.append(['time', 'Te-ft2', 'dTe-ft2', 'Ti-ft2', 'dTi-ft2', 'flux-Te-ft2', 'flux-Ti-ft2'])
active_t.append(['time', 'Te-ft3', 'dTe-ft3', 'Ti-ft3', 'dTi-ft3', 'flux-Te-ft3', 'flux-Ti-ft3'])
active_t.append(['time', 'Te-ft4', 'dTe-ft4', 'Ti-ft4', 'dTi-ft4', 'flux-Te-ft4', 'flux-Ti-ft4'])
active_t.append(['time', 'Te-ft5', 'dTe-ft5', 'Ti-ft5', 'dTi-ft5', 'flux-Te-ft5', 'flux-Ti-ft5'])
active_t.append(['time', 'Te-ft6', 'dTe-ft6', 'Ti-ft6', 'dTi-ft6', 'flux-Te-ft6', 'flux-Ti-ft6'])
active_t.append(['time', 'Te-ft7', 'dTe-ft7', 'Ti-ft7', 'dTi-ft7', 'flux-Te-ft7', 'flux-Ti-ft7'])
active_t.append(['time', 'Te-ft8', 'dTe-ft8', 'Ti-ft8', 'dTi-ft8', 'flux-Te-ft8', 'flux-Ti-ft8'])

In [8]:
# setup some column groups 
Te_cols = ['Te-ft1', 'Te-ft2', 'Te-ft3', 'Te-ft4', 'Te-ft5', 'Te-ft6', 'Te-ft7', 'Te-ft8']
Ti_cols = ['Ti-ft1', 'Ti-ft2', 'Ti-ft3', 'Ti-ft4', 'Ti-ft5', 'Ti-ft6', 'Ti-ft7', 'Ti-ft8']
dTe_cols = ['dTe-ft1', 'dTe-ft2', 'dTe-ft3', 'dTe-ft4', 'dTe-ft5', 'dTe-ft6', 'dTe-ft7', 'dTe-ft8']
dTi_cols = ['dTi-ft1', 'dTi-ft2', 'dTi-ft3', 'dTi-ft4', 'dTi-ft5', 'dTi-ft6', 'dTi-ft7', 'dTi-ft8']
Te_flux_cols = ['flux-Te-ft1', 'flux-Te-ft2', 'flux-Te-ft3', 'flux-Te-ft4', 
                'flux-Te-ft5', 'flux-Te-ft6', 'flux-Te-ft7', 'flux-Te-ft8'] 
Ti_flux_cols = ['flux-Ti-ft1', 'flux-Ti-ft2', 'flux-Ti-ft3', 'flux-Ti-ft4', 
                'flux-Ti-ft5', 'flux-Ti-ft6', 'flux-Ti-ft7', 'flux-Ti-ft8']
ne_cols = ['ne-ft1', 'ne-ft2', 'ne-ft3', 'ne-ft4', 'ne-ft5', 'ne-ft6', 'ne-ft7', 'ne-ft8']
dne_cols = ['dne-ft1', 'dne-ft2', 'dne-ft3', 'dne-ft4', 'dne-ft5', 'dne-ft6', 'dne-ft7', 'dne-ft8']
ne_flux_cols = ['flux-ne-ft1', 'flux-ne-ft2', 'flux-ne-ft3', 'flux-ne-ft4', 
                'flux-ne-ft5', 'flux-ne-ft6', 'flux-ne-ft7', 'flux-ne-ft8']
cp_Te_flux_cols = ['cp-flux-Te-ft1', 'cp-flux-Te-ft2', 'cp-flux-Te-ft3', 'cp-flux-Te-ft4', 
                   'cp-flux-Te-ft5', 'cp-flux-Te-ft6', 'cp-flux-Te-ft7', 'cp-flux-Te-ft8'] 
cp_Ti_flux_cols = ['cp-flux-Ti-ft1', 'cp-flux-Ti-ft2', 'cp-flux-Ti-ft3', 'cp-flux-Ti-ft4', 
                   'cp-flux-Ti-ft5', 'cp-flux-Ti-ft6', 'cp-flux-Ti-ft7', 'cp-flux-Ti-ft8']
cp_ne_flux_cols = ['cp-flux-ne-ft1', 'cp-flux-ne-ft2', 'cp-flux-ne-ft3', 'cp-flux-ne-ft4', 
                   'cp-flux-ne-ft5', 'cp-flux-ne-ft6', 'cp-flux-ne-ft7', 'cp-flux-ne-ft8']

In [6]:
# setup some column groups 
dvdrho_cols = ['dvdrho1', 'dvdrho2', 'dvdrho3', 'dvdrho4', 'dvdrho5', 'dvdrho6', 'dvdrho7', 'dvdrho8']
surfaceA_cols = ['surfaceA1', 'surfaceA2', 'surfaceA3', 'surfaceA4', 
                 'surfaceA5', 'surfaceA6', 'surfaceA7', 'surfaceA8']
Te_source_int_cols = ['Te-source-int-ft1', 'Te-source-int-ft2', 'Te-source-int-ft3', 'Te-source-int-ft4', 
                      'Te-source-int-ft5', 'Te-source-int-ft6', 'Te-source-int-ft7', 'Te-source-int-ft8'] 
Ti_source_int_cols = ['Ti-source-int-ft1', 'Ti-source-int-ft2', 'Ti-source-int-ft3', 'Ti-source-int-ft4', 
                      'Ti-source-int-ft5', 'Ti-source-int-ft6', 'Ti-source-int-ft7', 'Ti-source-int-ft8']
E_source_int_cols = ['E-source-int-ft1', 'E-source-int-ft2', 'E-source-int-ft3', 'E-source-int-ft4', 
                     'E-source-int-ft5', 'E-source-int-ft6', 'E-source-int-ft7', 'E-source-int-ft8']
E_flux_cols = ['flux-E-ft1', 'flux-E-ft2', 'flux-E-ft3', 'flux-E-ft4', 
               'flux-E-ft5', 'flux-E-ft6', 'flux-E-ft7', 'flux-E-ft8']
cp_E_flux_cols = ['cp-flux-E-ft1', 'cp-flux-E-ft2', 'cp-flux-E-ft3', 'cp-flux-E-ft4', 
                  'cp-flux-E-ft5', 'cp-flux-E-ft6', 'cp-flux-E-ft7', 'cp-flux-E-ft8']

## Add some columns for comparing the "power-balance" transport coefficients

In [7]:
AUG_gem0_pb[E_source_int_cols] = np.array(AUG_gem0_pb[Te_source_int_cols]) + np.array(AUG_gem0_pb[Ti_source_int_cols])
AUG_gem0_pb[Te_flux_cols] = np.array(AUG_gem0_pb[Te_source_int_cols]) ## / np.array(AUG_gem0_pb[surfaceA_cols])
AUG_gem0_pb[Ti_flux_cols] = np.array(AUG_gem0_pb[Ti_source_int_cols]) ## / np.array(AUG_gem0_pb[surfaceA_cols])
AUG_gem0_pb[E_flux_cols] = np.array(AUG_gem0_pb[E_source_int_cols]) ## / np.array(AUG_gem0_pb[surfaceA_cols])
#AUG_gem0_wf[E_flux_cols] = np.array(AUG_gem0_wf[Te_flux_cols]) + np.array(AUG_gem0_wf[Ti_flux_cols])
AUG_gem0_wf[E_flux_cols] = np.array(AUG_gem0_wf[cp_Te_flux_cols]) + np.array(AUG_gem0_wf[cp_Ti_flux_cols])


KeyError: "None of [Index(['E-source-int-ft1', 'E-source-int-ft2', 'E-source-int-ft3',\n       'E-source-int-ft4', 'E-source-int-ft5', 'E-source-int-ft6',\n       'E-source-int-ft7', 'E-source-int-ft8'],\n      dtype='object')] are in the [columns]"

In [ ]:
AUG_gem0_pb[E_flux_cols].describe().T

## Plot some of the "power-balance" columns

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(24,24))
AUG_gem0_pb.plot(x='time', y=dvdrho_cols, ax=ax[0,0])
AUG_gem0_pb.plot(x='time', y=surfaceA_cols, ax=ax[0,1])
AUG_gem0_pb.plot(x='time', y=Te_source_int_cols, ax=ax[1,0])
AUG_gem0_pb.plot(x='time', y=Ti_source_int_cols, ax=ax[1,1])
AUG_gem0_pb.plot(x='time', y=Te_flux_cols, ax=ax[2,0])
AUG_gem0_pb.plot(x='time', y=Ti_flux_cols, ax=ax[2,1])
AUG_gem0_pb.plot(x='time', y=E_source_int_cols, ax=ax[3,0])
AUG_gem0_pb.plot(x='time', y=E_flux_cols, ax=ax[3,1])
plt.suptitle('AUG_gem0_pb');

## Flux tube electron and ion energy fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (1e4,1e5)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (1e4,1e5)}],
    logy=True)

## Flux tube Te & Ti

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}],
    [{"cols": Te_cols, "label": 'electron temperature [eV]', "limit": (200,3500)},
     {"cols": Ti_cols, "label": 'ion temperature [eV]', "limit": (200,2500)}])

## Flux tube dTe & dTi

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}],
    [{"cols": dTe_cols, "label": 'electron temperature gradient [eV/m]', "limit": (-8000,0)},
     {"cols": dTi_cols, "label": 'ion temperature gradient [eV/m]', "limit": (-5000,0)}],
    logy=False)

## Flux tube ne & dne (not evolved so should be pretty much constant)

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}],
    [{"cols": ne_cols, "label": 'electron density [m^{-3}]', "limit": (3.5e19,4.5e19)},
     {"cols": dne_cols, "label": 'electron density gradient [m^{-3}/m]', "limit": (-0.65e19,-0.45e19)}],
    logy=False)

## Flux tube ne fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}],
    [{"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1.5e20,0e20)},
     {"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1.5e20,0e20)}],
    logy=False)

## Summed energy fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem0_wf, "style": '-'}, {"data": AUG_gem0_pb, "style": ':'}],
    [{"cols": E_flux_cols, "label": 'summed energy flux density [W m^{-2}]', "limit": (0e4,6e6)}],
    logy=False, figsize=(24,16))

## Power balance ratios

In [ ]:
plt.figure(figsize=(24,16))
plt.plot(AUG_gem0_wf.time, AUG_gem0_wf[E_flux_cols]/AUG_gem0_pb[E_flux_cols])
plt.legend(loc=0, labels=E_flux_cols, ncol=4)
plt.xlabel('Time'); plt.ylabel('wf/pb energy flux'); plt.ylim(0.99, 1.01); plt.suptitle('Power Balance Ratios');

# The End

In [ ]:
import utility, importlib
importlib.reload(utility)
from utility import _ewma, plot_smoothed, plot_unsmoothed